
# **Mental Health Assistant Chatbot**

This project aims to create an Emotionally Intelligent Chatbot for Mental Health support. It combines two key components: an `Emotion Classifier Model` created by [SamLowe](https://huggingface.co/SamLowe/roberta-base-go_emotions) and the `Llama language model` created by [NousResearch](https://huggingface.co/NousResearch/Llama-2-7b-chat-hf). The emotion classifier analyzes user input to determine their emotional state, providing detailed scores for emotions like sadness, joy, and anger. This emotional context is then integrated into prompts for `Llama 2`, enabling it to generate responses that are both contextually relevant and emotionally appropriate.
The chatbot is designed to detect when users are feeling down and provide supportive, empathetic responses. It acknowledges the user's emotional state, addresses their concerns, and offers encouragement. This system aims to provide a supportive presence for users, especially during times of emotional distress.

Key features include:
1.   Real-time emotion analysis of user input
2.   Context-aware response generation using Llama 2
3.   Tailored support based on detected emotional states
4.   Conversational communication style

While not a replacement for professional mental health care, this chatbot serves as a supplementary tool for emotional support and a potential first point of contact for those seeking help. The project shows the importance of responsible AI implementation in mental health applications, including appropriate disclaimers and safeguards.

## **1. Installing Dependencies**
In this section, we install the necessary libraries for working with the Hugging Face Transformers library, Llama 2, and PyTorch.
We will install additional utilities such as `huggingface_hub` for managing models and `sentencepiece` for tokenization.

In [ ]:
!pip install -q accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers huggingface_hub

## **2. Importing Libraries**
Here, we import the necessary Python libraries for handling language models, tokenization, and model authentication.
- `transformers`: Provides access to pre-trained language models.
- `AutoModelForCausalLM`, `AutoTokenizer`: Used for loading Llama 2.
- `torch`: PyTorch framework for deep learning.
- `pipeline`: Utility for setting up pipelines like the emotion classifier.

In [2]:
import pandas as pd
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
import torch
from google.colab import userdata

## **3. Hugging Face Authentication**
We authenticate with Hugging Face using a pre-generated token stored in Colab.
This allows us to access and load models hosted on Hugging Face.

In [ ]:
# Hugging Face Authentication
login(token=userdata.get('hugging_token_secret'))

## **4. Initializing the Models**
We start loading the models using pipelines from Hugging Face.


In [ ]:
# Loading the our emotional classifier model by passing the parameters

emotion_classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

In [ ]:
# Initialize the Llama 2 model and tokenizer
model_id = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

# Initialize the pipeline using Hugging Face pipeline
llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=1024,  # Adjust max_length as needed
)

Testing our loaded models

In [ ]:
prediction = emotion_classifier("I feel disgusted.", )
print(prediction[0])

In [ ]:
prediction2 = llama_pipeline("What is Mental Health?", max_length=1024, do_sample=True)
print(prediction2[0]['generated_text'])

## **5. Creation of the User input processing function**
We start by creating the main function `process_user_message` for analyzng and processing the text provided by the user together with its utility functions.

Below is a detailed explanation of the utility functions:


 1. `get_primary_emotion function`:  This function determines the most prominent emotion based on the highest score.

2.  `create_emotion_aware_prompt function`:
This function crafts a prompt for Llama 2 that includes:
    *  *The user's original input.*
    *  *A detailed breakdown of their emotional state.*
    *  *Guidelines for generating an appropriate response.*

3.  `generate_emotion_context function:`
This function creates a detailed description of the user's emotional state, including the primary emotion and a breakdown of all emotion scores.




In [7]:
def process_user_message(user_input):
    # Step 1: Process user input
    emotion_scores = emotion_classifier(user_input)[0]

    # Step 2: Analyze emotions
    primary_emotion = get_primary_emotion(emotion_scores)

    # Step 3: Generate response using Llama
    prompt = create_emotion_aware_prompt(user_input, emotion_scores, primary_emotion)
    response = llama_pipeline(prompt, max_length=1024, do_sample=True)[0]['generated_text']

    # Ensure the response doesn't redundantly include the question or incorrectly repeat "Answer"
    response = response.replace(f"{prompt}", "").strip()

    # Step 4: Deliver response
    return response


## Utility functions for emotional intergrations with LLM
def get_primary_emotion(emotion_scores):
    return max(emotion_scores, key=lambda x: x['score'])['label']

def create_emotion_aware_prompt(user_input, emotion_scores, primary_emotion):
    emotion_context = generate_emotion_context(emotion_scores, primary_emotion)

    return f"""
    The user has expressed the following: "{user_input}"

    Emotional analysis:
    {emotion_context}

    Please provide a response that:
    1. Acknowledges and validates the user's emotional state
    2. Addresses the content of their message
    3. Offers support or encouragement appropriate to their emotional state
    4. Maintains a conversational and empathetic tone

    Generate a response:
    """

def generate_emotion_context(emotion_scores, primary_emotion):
    context = f"The user's primary emotion appears to be {primary_emotion}.\n"
    context += "Emotional state breakdown:\n"

    for emotion in emotion_scores:
        context += f"- {emotion['label']}: {emotion['score']:.4f}\n"

    return context

## **6. Testing our Text Processing function**

In [ ]:
#Example usage
user_input = "I am fed up with the constant disregard for my needs and feelings, and I demand respect and acknowledgment from those who consistently disregard my boundaries."
response = process_user_message(user_input)
print(response)

## **6. Creating User Interface Chatbot using Gradio**

Step 1:   We start by installing gradio dependancies.

Step 2:  We create a function for gradio to handle user inputs whereby the function processes the user input and returns the response

Step 3:   We create the Gradio interface and launch it to generate the ChatBot.

In [ ]:
!pip -q install gradio

In [10]:
import gradio as gr

def gradio_chat_interface(user_input):

  response = process_user_message(user_input)
  return response


In [11]:
# Create a Gradio Interface
interface = gr.Interface(
    fn=gradio_chat_interface,
    inputs="text",
    outputs="text",
    title="Llama 2 Mental Health Chat Assistant",
    description="Let us talk Today",
)

In [ ]:
# Launch the Gradio Interface
interface.launch()